In [ ]:
import warnings
warnings.filterwarnings("ignore")
from parameterize import *
########################################################################################################################################################################################################
system = PrepareQMMM(init_pdb = "test_system.pdb", cleaned_pdb = "system.pdb", guest_init_pdb = "guest_init.pdb", host_pdb = "host.pdb", guest_resname = "BEN", guest_pdb = "guest_init_II.pdb", guest_xyz = "guest_coord.txt", distance = 6.0, residue_list = "residue_list.txt", host_qm_atoms = "host_qm.txt", host_mm_atoms = "host_mm.txt", host_qm_pdb = "host_qm.pdb", host_mm_pdb = "host_mm.pdb", qm_pdb = "qm.pdb", mm_pdb = "mm.pdb", host_mm_region_I_atoms = "host_mm_region_I.txt", host_mm_region_II_atoms = "host_mm_region_II.txt", host_mm_region_I_pdb = "host_mm_region_I.pdb", host_mm_region_II_pdb = "host_mm_region_II.pdb", num_residues = 2)
########################################################################################################################################################################################################
qm_guest = PrepareGaussianGuest(guest_pdb = "guest_init_II.pdb", n_processors = 12, memory = 50, charge = 1, multiplicity = 1, functional = "B3LYP", basis_set = "6-31G", optimisation = "OPT", frequency = "FREQ", add_keywords_I = "Integral=(Grid=UltraFine)", add_keywords_II = "Pop(MK,ReadRadii)", add_keywords_III = "IOp(6/33=2,6/42=6)", gauss_out_file = "guest.out", fchk_out_file = "guest_fchk.out")
########################################################################################################################################################################################################
qm_system = PrepareGaussianHostGuest(guest_pdb = "guest_init_II.pdb", host_qm_pdb = "host_qm.pdb", n_processors = 12, memory = 50, charge = 1, multiplicity = 1, functional = "B3LYP", basis_set = "6-31G", optimisation = "", frequency = "", add_keywords_I = "Integral=(Grid=UltraFine)", add_keywords_II = "Pop(MK,ReadRadii)", add_keywords_III = "IOp(6/33=2,6/42=6)", gauss_system_out_file = "system_qm.out", fchk_system_out_file = "system_qm_fchk.out", host_guest_input = "host_guest.com", qm_guest_charge_parameter_file = "guest_qm_surround_charges.txt", qm_host_charge_parameter_file = "host_qm_surround_charges.txt", qm_guest_atom_charge_parameter_file = "guest_qm_atom_surround_charges.txt")
########################################################################################################################################################################################################
params_guest = ParameterizeGuest(xyz_file = "guest_coords.xyz", coordinate_file = "guest_coordinates.txt", unprocessed_hessian_file = "guest_unprocessed_hessian.txt", bond_list_file = "guest_bond_list.txt", angle_list_file = "guest_angle_list.txt", hessian_file = "guest_hessian.txt", atom_names_file = "guest_atom_names.txt", bond_parameter_file = "guest_bonds.txt", vibrational_scaling = 1.00, angle_parameter_file = "guest_angles.txt", charge_parameter_file = "guest_charges.txt", guest_pdb = "guest_init_II.pdb", proper_dihedral_file = "proper_dihedrals.txt")
########################################################################################################################################################################################################
qm_host = PrepareGaussianHost(host_qm_pdb = "host_qm.pdb", n_processors = 12, memory = 50, charge = 0, multiplicity = 1, functional = "B3LYP", basis_set = "6-31G", optimisation = "OPT", frequency = "FREQ", add_keywords_I = "Integral=(Grid=UltraFine)", add_keywords_II = "Pop(MK,ReadRadii)", add_keywords_III = "IOp(6/33=2,6/42=6)", gauss_out_file = "host_qm.out", fchk_out_file = "host_qm_fchk.out")
########################################################################################################################################################################################################
params_host = ParameterizeHost(xyz_file = "host_qm_coords.xyz", coordinate_file = "host_qm_coordinates.txt", unprocessed_hessian_file = "host_qm_unprocessed_hessian.txt", bond_list_file = "host_qm_bond_list.txt", angle_list_file = "host_qm_angle_list.txt", hessian_file = "host_qm_hessian.txt", atom_names_file = "host_qm_atom_names.txt", bond_parameter_file = "host_qm_bonds.txt", vibrational_scaling = 1.00, angle_parameter_file = "host_qm_angles.txt", charge_parameter_file = "host_qm_surround_charges.txt", host_qm_pdb = "host_qm.pdb")
########################################################################################################################################################################################################
system_guest = GuestAmberXMLAmber(system_pdb = "guest_init_II.pdb", system_mol2 = "guest.mol2", system_in = "guest.in", charge = 1, system_frcmod = "guest.frcmod", prmtop_system = "guest.prmtop", inpcrd_system = "guest.inpcrd", system_leap = "guest.leap", system_xml = "guest_init.xml",  system_smi =  "guest.smi",  system_sdf =  "guest.sdf", system_init_sdf = "guest_init.sdf", num_charge_atoms = 1, index_charge_atom_1 = 9, charge_atom_1= 1, index_charge_atom_2 = " ", charge_atom_2 = " ", charge_parameter_file = "guest_charges.txt", system_qm_pdb = "guest_init_II.pdb", bond_parameter_file = "guest_bonds.txt", angle_parameter_file = "guest_angles.txt", system_qm_params_file = "guest_qm_params.txt", reparameterised_intermediate_system_xml_file = "guest_intermediate_reparameterised.xml", system_xml_non_bonded_file = "guest_xml_non_bonded.txt", system_xml_non_bonded_reparams_file = "guest_xml_non_bonded_reparams.txt", reparameterised_system_xml_file = "guest_reparameterised.xml", non_reparameterised_system_xml_file = "guest_init.xml", prmtop_system_non_params = "guest_non_params.prmtop", inpcrd_system_non_params = "guest_non_params.inpcrd", prmtop_system_params = "guest_params.prmtop", inpcrd_system_params = "guest_params.inpcrd", load_topology = "openmm")
########################################################################################################################################################################################################
system_host = HostAmberXMLAmber(system_pdb = "host.pdb", system_xml = "host.xml", sim_output = "sim_output.pdb", sim_steps = 1000, charge_parameter_file = "host_qm_surround_charges.txt", system_qm_pdb = "host_qm.pdb", bond_parameter_file = "host_qm_bonds.txt", angle_parameter_file = "host_qm_angles.txt", system_qm_params_file = "host_qm_params.txt", reparameterised_intermediate_system_xml_file = "host_intermediate_reparameterised.xml", system_xml_non_bonded_file = "host_xml_non_bonded.txt", system_xml_non_bonded_reparams_file = "host_xml_non_bonded_reparams.txt", reparameterised_system_xml_file = "host_reparameterised.xml", non_reparameterised_system_xml_file = "host.xml", prmtop_system_non_params = "host_non_params.prmtop", inpcrd_system_non_params = "host_non_params.inpcrd", prmtop_system_params = "host_params.prmtop", inpcrd_system_params = "host_params.inpcrd", load_topology = "openmm")
########################################################################################################################################################################################################
guest_openmm = RunOpenMMSims(system_prmtop = "guest_params.prmtop", system_inpcrd = "guest_params.inpcrd", system_pdb = "guest_init_II.pdb", system_output =  "sim_output.pdb", sim_steps = 1000)
########################################################################################################################################################################################################
host_openmm = RunOpenMMSims(system_prmtop = "host_params.prmtop", system_inpcrd = "host_params.inpcrd", system_pdb = "host.pdb", system_output =  "sim_output.pdb", sim_steps = 1000)
########################################################################################################################################################################################################
merge_host_guest = MergeHostGuestTopology(host_prmtop = "host_params.prmtop", guest_prmtop = "guest_params.prmtop", host_inpcrd = "host_params.inpcrd", guest_inpcrd = "guest_params.inpcrd", system_prmtop = "system_params.prmtop", system_inpcrd = "system_params.inpcrd")
########################################################################################################################################################################################################
system_openmm = RunOpenMMSims(system_prmtop = "system_params.prmtop", system_inpcrd = "system_params.inpcrd", system_pdb = "system.pdb", system_output =  "sim_output.pdb", sim_steps = 1000)
########################################################################################################################################################################################################
guest_torsion_params = TorsionDriveSims(reparameterised_system_xml_file = "guest_reparameterised.xml", torsion_xml_file = "guest_torsion_xml.txt", xyz_file = "guest_coords.xyz", psi_input_file = "torsion_drive_input.dat", memory = 50, charge = 1, multiplicity = 1, basis_set = "STO-3G", functional = "BLYP", iterations = 2000, method_torsion_drive = "native_opt", system_bonds_file = "guest_bonds.txt", tor_dir = "torsion_dir", dihedral_text_file = "dihedrals.txt", template_pdb = "guest_init_II.pdb", torsion_drive_run_file = "run_command", dihedral_interval = 15, engine = "psi4", energy_threshold = 0.001)
########################################################################################################################################################################################################
guest_write_params = TorsionDriveParams(tor_dir = "torsion_dir", reparameterized_torsional_params_file = "reparameterized_torsional_params.txt", psi_input_file = "torsion_drive_input.dat", xyz_file = "torsion_drive_input.xyz", coords_file = "torsion_drive_input.txt", template_pdb = "guest_init_II.pdb", system_pdb = "torsion_drive_input.pdb", system_sdf = "torsion_drive_input.sdf", system_xml = "torsion_drive_input.xml", qm_scan_file = "scan.xyz", load_topology = "openmm", method = "L-BFGS-B", dihedral_text_file = "dihedrals.txt", system_init_sdf = "torsion_drive_input_init.sdf", num_charge_atoms = 1, index_charge_atom_1 = 9, charge_atom_1 = 1)
########################################################################################################################################################################################################

## Initial files needed : Host-Guest PDB

## Step I : Defining QM and MM Regions

In [ ]:
system.clean_up()
system.create_host_guest()
system.realign_guest()
system.get_guest_coord()
system.get_qm_resids()
system.get_host_qm_mm_atoms()
system.save_host_pdbs()
system.get_host_mm_region_atoms()
system.save_host_mm_regions_pdbs()
system.get_qm_mm_regions()

## Step II : Guest QM Calculation 

In [ ]:
qm_guest.write_input()
#qm_guest.run_gaussian()
#qm_guest.get_fchk()

## Step III : QM Calculation of the Host-Guest region for QM - Derived Charges

In [ ]:
qm_system.write_input()
#qm_system.run_gaussian()
#qm_system.get_fchk()
qm_system.get_qm_host_guest_charges()

## Step IV : Guest Re - Parameterization

In [ ]:
params_guest.get_xyz()
params_guest.get_unprocessed_hessian()
params_guest.get_bond_angles()
params_guest.get_hessian()
params_guest.get_atom_names()
params_guest.get_bond_angle_params()
params_guest.get_charges()
params_guest.get_proper_dihedrals()

## Step V : Host (Residues / Region Surrounding the Guest) QM Calculation 

In [ ]:
qm_host.write_input()
#qm_host.run_gaussian()
#qm_host.get_fchk()

## Step VI : Host (Residues / Region Surrounding the Guest)  Re - Parameterization

In [ ]:
params_host.get_xyz()
params_host.get_unprocessed_hessian()
params_host.get_bond_angles()
params_host.get_hessian()
params_host.get_atom_names()
params_host.get_bond_angle_params()
params_host.get_charges()

## Step VII : Generation of Re - Parameterized .prmtop & .inpcrd Files for Guest

In [ ]:
system_guest.generate_xml_from_charged_pdb_sdf()
system_guest.write_system_params()
system_guest.write_reparameterised_system_xml()
system_guest.save_amber_params()
system_guest.analyze_diff_energies()

## Step VIII : Generation of Re - Parameterized .prmtop & .inpcrd Files for the Host

In [ ]:
system_host.serialize_system()
system_host.write_system_params()
system_host.write_reparameterised_system_xml()
system_host.save_amber_params()
system_host.analyze_diff_energies()

## Step IX : Re - Parameterization of the torsion angles for the Guest (In Progress)

## Step X : OpenMM Test - Run of the Newly Generated Parameter Files for the System

In [ ]:
guest_openmm.run_openmm_prmtop_inpcrd()
guest_openmm.run_openmm_prmtop_pdb()
host_openmm.run_openmm_prmtop_inpcrd()
host_openmm.run_openmm_prmtop_pdb()
merge_host_guest.merge_topology_files()
system_openmm.run_openmm_prmtop_inpcrd()
system_openmm.run_openmm_prmtop_pdb()

### Step XI : Torsions

In [ ]:
guest_torsion_params.write_torsion_drive_run_file()
guest_torsion_params.write_tor_params_txt()
guest_torsion_params.write_psi4_input()
guest_torsion_params.create_non_H_bonded_torsion_drive_dir()
guest_torsion_params.run_torsion_sim()

In [ ]:
guest_write_params.write_reparams_torsion_lines_charged()

In [ ]:
write_torsional_reparams(reparameterised_system_xml_file = "guest_reparameterised.xml", reparameterized_torsional_params_file = "reparameterized_torsional_params.txt", reparameterised_torsional_system_xml_file = "guest_torsional_reparameterized.xml")

### Step XII : Solvation and Ion addition to the system